In [ ]:
# Installing required libs
!pip install -U transformers
!pip install -U datasets
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate
!pip install evaluate
!pip install rouge_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Un

In [ ]:
# Download the zip file and extract
!wget -O master.zip https://github.com/ryanzhumich/AESLC/archive/refs/heads/master.zip
%rm -rf /content/master/

import zipfile
import os

def extract_zip(zip_path, extract_to):
    # Ensure the extraction directory exists
    os.makedirs(extract_to, exist_ok=True)

    # Open the ZIP file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all the contents to the specified directory
        zip_ref.extractall(extract_to)

zip_path = '/content/master.zip'
extract_to = '/content/master'
extract_zip(zip_path, extract_to)

--2024-07-10 15:16:07--  https://github.com/ryanzhumich/AESLC/archive/refs/heads/master.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/ryanzhumich/AESLC/zip/refs/heads/master [following]
--2024-07-10 15:16:07--  https://codeload.github.com/ryanzhumich/AESLC/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.9
Connecting to codeload.github.com (codeload.github.com)|140.82.114.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [    <=>             ]  11.11M  13.8MB/s    in 0.8s    

2024-07-10 15:16:09 (13.8 MB/s) - ‘master.zip’ saved [11650497]



In [ ]:
# Load train, dev and test sets into lists
import glob

train_directory_path  = '/content/master/AESLC-master/enron_subject_line/train'
dev_directory_path  = '/content/master/AESLC-master/enron_subject_line/dev'
test_directory_path  = '/content/master/AESLC-master/enron_subject_line/test'

def load_files(directory, has_annotations=False):
    files = glob.glob(os.path.join(directory, "*"))
    data = []
    for file in files:
        with open(file, 'r', encoding='utf-8') as file:
            content = file.read()
        email_body = ""
        subject_line = ""
        email_body, subjects = content.split('@subject\n', 1)
        annotations = []
        if has_annotations:
          subjects = subjects.replace("@ann0", "@subject\n").replace("@ann1", "@subject\n").replace("@ann2", "@subject\n")
          subject_line, annotation_1, annotation_2, annotation_3 = subjects.split('@subject\n')
          subject_line = subject_line.strip()
          annotations = [annotation_1.strip(), annotation_2.strip(), annotation_3.strip()]
          data.append((email_body, subject_line, annotations))
        else:
          subject_line = subjects.strip()
          data.append((email_body, subject_line))
    return data

# Load the data
dev_data = load_files(dev_directory_path, has_annotations=True)
train_data = load_files(train_directory_path)
test_data = load_files(test_directory_path, has_annotations=True)

# Debugging: Check dataset lengths
print(f"Train dataset length: {len(train_data)}")
print(f"Dev dataset length: {len(dev_data)}")
print(f"Test dataset length: {len(test_data)}")


Train dataset length: 14436
Dev dataset length: 1960
Test dataset length: 1906


In [ ]:
train_data[:2]

[("Attached is your reservation for Rome for September 29, 2001.\nThis room is  charged to your Amex account but the reservation is in my name.\nPrint this attachment out and take it with you, since it has the  confirmation # on it.\nYou will have a king, non-smoking, deluxe room at  $189.00 per night, including breakfast.\nThis probably isn't in the downtown area, since it is supposedly a golf  resort.\nPaul said he would find out the exact location for us.\nIf you are able to locate something better, feel free.\nI could find several  hotels on Travelocity that had availability, but the phone # wasn't provided  and you can only book one room at a time.\nThe cancellation policy on this hotel is good -- we just have to cancel by\n\n",
  'Rome reservations'),
 ("Pam Daily would like for you to give her a call about moving to Steve's old office.\nShe said she was getting the information second hand from Audry and found it quite hard to believe from her, so she would like to hear this from

In [ ]:
train_data[0][1], dev_data[0]

('Rome reservations',
 ("This message is only relevant for those going to UBS.\n...\nIt appears that the following email may not have been clear.\nYou have to copy all documents yourself.\nThis is my cheat sheet if it helps:    H Drives: 1.\nEach person will have to move any data over personally from the old H drive (which is now Z:) drive to their new H drive.\n2.\nThe Z: drive (old H:) will be removed from UBS employees on Thursday night.\nO Drives: 1.\nFor the old O or M drives go to O:\\ESTATE\\ O (or M ?\npending location of data) and copy to the new common O.\nThe Estate O and M will be removed from UBS employees on Thursday night.\nDo not delete anything and only copy information which is complies with the policy.\nPlease make sure that those people who are not in the office are contacted and comply.\nLouise      To:\t\tAll Enron Employees Transferring to UBS Warburg Energy (or its affiliates)  Under the terms of the various agreements among certain Enron entities and UBS AG, ce

In [ ]:
from google.colab import userdata
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

SecretNotFoundError: Secret HF_TOKEN does not exist.

In [ ]:
# Import model tokenizer and define global variables
import torch
import pprint
import evaluate
import numpy as np

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset

MODEL = 't5-small'
BATCH_SIZE = 12
NUM_PROCS = 4
EPOCHS = 2
OUT_DIR = 'results_t5base'
MAX_LENGTH = 512 # Maximum context length to consider while preparing dataset.

tokenizer = T5Tokenizer.from_pretrained(MODEL)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Create Tokenized data loaders for Train, Dev and Test sets

from torch.utils.data import Dataset
class EmailDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        email_body, subject_line = self.data[index][:2]
        prompt = f"summarize: {email_body}"

        # Tokenize the prompt and the target subject line
        inputs = self.tokenizer(prompt, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
        labels = self.tokenizer(subject_line, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")

        # Squeeze the tensors to remove the batch dimension
        input_ids = inputs["input_ids"].squeeze(0)
        attention_mask = inputs["attention_mask"].squeeze(0)
        labels = labels["input_ids"].squeeze(0)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

train_data_tokenized = EmailDataset(train_data, tokenizer )
dev_data_tokenized = EmailDataset(dev_data, tokenizer)
test_data_tokenized = EmailDataset(test_data, tokenizer)

In [ ]:
for email in train_data_tokenized:
  print(email)
  break

{'input_ids': tensor([21603,    10, 28416,    15,    26,    19,    39, 13019,    21,  7332,
           21,  1600, 14405,  4402,     5,   100,   562,    19,  4977,    12,
           39,   736,   994,   905,    68,     8, 13019,    19,    16,    82,
          564,     5,  6763,    48, 11352,    91,    11,   240,    34,    28,
           25,     6,   437,    34,    65,     8,  9973,  1713,    30,    34,
            5,   148,    56,    43,     3,     9,     3,  1765,     6,   529,
           18,     7,    51,    32,  1765,     6,     3, 27747,   562,    44,
         1970,  3914,  4200,   399,   706,     6,   379,  3688,     5,   100,
         1077,    19,    29,    31,    17,    16,     8,  7092,   616,     6,
          437,    34,    19,     3, 24309,     3,     9,  4319,  4631,     5,
         1838,   243,     3,    88,   133,   253,    91,     8,  2883,  1128,
           21,   178,     5,   156,    25,    33,     3,   179,    12,  9586,
          424,   394,     6,   473,   339,     5, 

In [ ]:
# Load the Model into device and check parameters
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

60,506,624 total parameters.
60,506,624 training parameters.


In [ ]:
inputs = tokenizer.encode(
    "summarize: " + train_data[0][0],
    return_tensors='pt',
    max_length=512,
    truncation=True
)
inputs = inputs.to(device)
inputs

tensor([[21603,    10, 28416,    15,    26,    19,    39, 13019,    21,  7332,
            21,  1600, 14405,  4402,     5,   100,   562,    19,  4977,    12,
            39,   736,   994,   905,    68,     8, 13019,    19,    16,    82,
           564,     5,  6763,    48, 11352,    91,    11,   240,    34,    28,
            25,     6,   437,    34,    65,     8,  9973,  1713,    30,    34,
             5,   148,    56,    43,     3,     9,     3,  1765,     6,   529,
            18,     7,    51,    32,  1765,     6,     3, 27747,   562,    44,
          1970,  3914,  4200,   399,   706,     6,   379,  3688,     5,   100,
          1077,    19,    29,    31,    17,    16,     8,  7092,   616,     6,
           437,    34,    19,     3, 24309,     3,     9,  4319,  4631,     5,
          1838,   243,     3,    88,   133,   253,    91,     8,  2883,  1128,
            21,   178,     5,   156,    25,    33,     3,   179,    12,  9586,
           424,   394,     6,   473,   339,     5,  

In [ ]:
summary_ids = model.generate(
  train_data_tokenized[0]['input_ids'].unsqueeze(0).to(device),
  #inputs,
  max_length=50,
  num_beams=5,
  early_stopping=True,
)
print(f"email_body : {train_data[0][0]}")
print(f"Actual Subject : {train_data[0][1]}")
print(f"Generated Subject : {tokenizer.decode(summary_ids[0], skip_special_tokens=True)}")

email_body : Attached is your reservation for Rome for September 29, 2001.
This room is  charged to your Amex account but the reservation is in my name.
Print this attachment out and take it with you, since it has the  confirmation # on it.
You will have a king, non-smoking, deluxe room at  $189.00 per night, including breakfast.
This probably isn't in the downtown area, since it is supposedly a golf  resort.
Paul said he would find out the exact location for us.
If you are able to locate something better, feel free.
I could find several  hotels on Travelocity that had availability, but the phone # wasn't provided  and you can only book one room at a time.
The cancellation policy on this hotel is good -- we just have to cancel by


Actual Subject : Rome reservations
Generated Subject : this room is charged to your Amex account but the reservation is in my name. you will have a king, non-smoking, deluxe room at $189.00 per night. this probably isn'


In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak.
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [ ]:
# Model Training
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    #warmup_steps=100,
    #weight_decay=0.01,
    logging_dir=OUT_DIR,
    #logging_steps=10,
    evaluation_strategy='epoch',#'steps',
    #eval_steps=2000,
    save_strategy='epoch',
    save_total_limit=1,
    report_to='tensorboard',
    learning_rate=0.0001,
    dataloader_num_workers=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data_tokenized,
    eval_dataset=dev_data_tokenized,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)

history = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Gen Len
1,0.049300,0.047786,0.333700,0.149500,0.330100,8.301000
2,0.047000,0.047354,0.339000,0.154500,0.335900,8.301000


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
# Save the trained model
tokenizer.save_pretrained(OUT_DIR)
!zip -r {OUT_DIR} {OUT_DIR}

  adding: results_t5base/ (stored 0%)
  adding: results_t5base/checkpoint-7218/ (stored 0%)
  adding: results_t5base/checkpoint-7218/scheduler.pt (deflated 56%)
  adding: results_t5base/checkpoint-7218/config.json (deflated 62%)
  adding: results_t5base/checkpoint-7218/model.safetensors (deflated 8%)
  adding: results_t5base/checkpoint-7218/generation_config.json (deflated 29%)
  adding: results_t5base/checkpoint-7218/training_args.bin (deflated 52%)
  adding: results_t5base/checkpoint-7218/trainer_state.json (deflated 70%)
  adding: results_t5base/checkpoint-7218/rng_state.pth (deflated 25%)
  adding: results_t5base/checkpoint-7218/optimizer.pt (deflated 7%)
  adding: results_t5base/special_tokens_map.json (deflated 85%)
  adding: results_t5base/events.out.tfevents.1720626970.5b70f571f54c.236.1 (deflated 63%)
  adding: results_t5base/added_tokens.json (deflated 83%)
  adding: results_t5base/events.out.tfevents.1720626057.5b70f571f54c.236.0 (deflated 69%)
  adding: results_t5base/token

In [ ]:
zip_path = '/content/results_t5base.zip'
extract_to = '/content/results_t5base'
extract_zip(zip_path, extract_to)

In [ ]:
# load the trained model

model_path = f"/content/{OUT_DIR}/checkpoint-7218"  # the path where you saved your model
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(OUT_DIR)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# verify on test data
for i in range(5):
    inputs = tokenizer.encode(
        "summarize: " + test_data[i][0],
        return_tensors='pt',
        max_length=512,
        truncation=True
    ).to(device)

    summary_ids = model.to(device).generate(
      inputs,
      max_length=10,
      num_beams=5,
      early_stopping=True,
    )
    print(f"email_body : {test_data[i][0]}")
    print(f"Actual Subject : {test_data[i][1]}")
    print(f"Generated Subject After training : {tokenizer.decode(summary_ids[0], skip_special_tokens=True)}")
    print('--------------------------------------------------------------------------------------------------------')

email_body : Just a reminder.
Make sure you are using Pinnacle West and not APS when you are putting these deals in for this counter party.
We've been getting a lot of disputes  related to this problem.
Thanks


Actual Subject : APS to Pinnacle West
Generated Subject After training : Pinnacle West
--------------------------------------------------------------------------------------------------------
email_body : Questions:  What does being short/long a position mean?
What is curve shift?
What affects curve shift?
What are the basic components of a P&L?
What is a swap?
What are different types of swaps?
What is basis?
What is MTM value?
What is Accrual value?
What do you consider stress in the workplace?
How do you handle stress?
Examples of a stressful situation?
What do you consider long work hours?
Some aren't technical.
I don't have any problem solving questions.
DG


Actual Subject : Technical Questions
Generated Subject After training : What is short/long?
-----------------------